# Import data
This is the code from the assignment to load the data.

In [1]:
import pandas as pd

dataset = pd.read_csv("dataset.csv")
baskets = dataset.groupby("user_id").product_id.apply(set).tolist()
baskets[:5]

[{5614842, 5766379},
 {5861791, 5894239},
 {5830270, 5830275},
 {5635117, 5751383, 5809910},
 {5767496, 5767497, 5891498}]

# Association rule mining algorithm
> Implement an association rule mining algorithm, or use an existing online implementation.
Show that you understand the method by describing its function (without using code) in your
report. Make sure you are able to get the confidence and support of any found association
rules.

> Run the association rule mining algorithm on the given dataset. At this point, use only the
user id and product id columns. What are the top 10 association rules in terms of support
your method finds? Also include the confidence of these rules. What can you say about the
number of items in these rules?

## Method 1
I started with the apyori implementation, but this also allowed empty baskets, which causes errors in the inspect function. You can just skip to my own implementation, which you can find under the "Method 4" subtitle.
Using code from [this article](https://www.section.io/engineering-education/apriori-algorithm-in-python/), with the apriori algorithm from [apyori](https://pypi.org/project/apyori/).

In [64]:
from apyori import apriori

# support: measures the number of times a particular item or combination of items occur in a dataset
# confidence: measures how likely the customer is to consume item2 given they have consumed item1
# lift: a metric that determines the strength of association between the best rules, confidence/support
# TODO: define own min_upport, min_confidence and min_lift
rule = apriori(transactions=baskets, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)

In [65]:
results = list(rule)

# putting output into a pandas dataframe
def inspect(output):
    for result in output:
        try:
            tuple(result[2][0][0])[0]
        except Exception as e:
            print(result)
            raise e
    lhs = [tuple(result[2][0][0])[0] for result in output]
    rhs = [tuple(result[2][0][1])[0] for result in output]
    support = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))

output_DataFrame = pd.DataFrame(inspect(results),
                                columns=['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])

output_DataFrame.nlargest(n=10, columns='Support')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,5677043,5697463,0.004187,0.329341,57.687425
2,5809912,5809910,0.003730,0.583333,25.459302
3,5814516,5814517,0.003730,0.875000,201.664474
1,5809911,5809910,0.003578,0.746032,32.560196


## Method 2
Using code from [this site](https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6), which uses the [apriori_python library](https://pypi.org/project/apriori-python/). This method is also very slow, so trying eclat insted.

In [73]:
from apriori_python import apriori
freqItemSet, rules = apriori(baskets, minSup=0.01, minConf=0.001)
print(freqItemSet)
print(rules)

[[5614842, 5766379], [5894239, 5861791], [5830275, 5830270], [5635117, 5809910, 5751383], [5767496, 5767497, 5891498]]
{1: {frozenset({5809910}), frozenset({5649236}), frozenset({5677043}), frozenset({5790689})}}
[]


## Method 3
Let's try running eclat on it. To do this, I'll use code from [this site](https://towardsdatascience.com/the-eclat-algorithm-8ae3276d2d17), which makes use of [pyECLAT](https://pypi.org/project/pyECLAT/). This method also gives an error.


In [3]:
data = pd.DataFrame(baskets)
data.head()

,0,1,2,3,4,5,6,7,8,9
0,5614842,5766379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5894239,5861791.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5830275,5830270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5635117,5809910.0,5751383.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5767496,5767497.0,5891498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# we are looking for itemSETS
# we do not want to have any individual products returned
min_n_products = 2

# we want to set min support to 7
# but we have to express it as a percentage
min_support = 7/len(baskets)

# we have no limit on the size of association rules
# so we set it to the longest transaction
max_length = max([len(x) for x in baskets])

In [76]:
from pyECLAT import ECLAT

# create an instance of eclat
my_eclat = ECLAT(data=data, verbose=True)

# fit the algorithm
rule_indices, rule_supports = my_eclat.fit(min_support=min_support,
                                           min_combination=min_n_products,
                                           max_combination=max_length)

100%|██████████| 11687/11687 [00:07<00:00, 1587.45it/s]


ValueError: Cannot index with multidimensional key

## Method 4
Enough libraries tried that didn't work, let's just implement eclat ourselves.
I based this implementation on the explanation from the lecture and [this explanation](https://www.geeksforgeeks.org/ml-eclat-algorithm/).


In [3]:
from tqdm import tqdm

def make_tid_dict(baskets):
    tid_dict = dict()
    for index, item_set in tqdm(enumerate(baskets), total=len(baskets), desc="Making tid list"):
        for item in item_set:
            frozen_set = frozenset({item})
            if frozen_set not in tid_dict:
                tid_dict[frozen_set] = set()
            tid_dict[frozen_set].add(index)
    return tid_dict

tid_dict = make_tid_dict(baskets)
list(tid_dict.items())[:5]

Making tid list: 100%|██████████| 13137/13137 [00:00<00:00, 364916.53it/s]


[(frozenset({5614842}),
  {0,
   152,
   1026,
   1470,
   1576,
   1691,
   1799,
   1814,
   1910,
   2027,
   2322,
   2694,
   3259,
   4929,
   5229,
   5693,
   6024,
   6594,
   6647,
   6702,
   6755,
   6980,
   7351,
   8335,
   9987,
   10223,
   13101}),
 (frozenset({5766379}), {0, 85, 4706, 4933, 6206, 6473, 8476, 9647, 12363}),
 (frozenset({5894239}), {1, 7278, 10877}),
 (frozenset({5861791}), {1, 3982, 7420}),
 (frozenset({5830275}), {2, 2588, 2855, 12148})]

In [4]:
from tqdm import tqdm
from itertools import permutations

min_support = 2

def combine_sets(tid_dict, min_support):
    new_tid_dict = dict()
    combs = list(permutations(tid_dict.items(), r=2))
    for dict_item1, dict_item2 in tqdm(combs, total=len(combs)):
        item_set1, tid_set1 = dict_item1
        item_set2, tid_set2 = dict_item2

        assert len(item_set1) == len(item_set2)
        assert len(tid_set1) >= min_support and len(tid_set2) >= min_support

        new_set = frozenset(item_set1 | item_set2) # Same for this union operator

        # Only search for sets one size larger
        if len(new_set) != len(item_set1)+1:
            continue

        new_set_items = tid_set1 & tid_set2 # Note: This intersection operator is new from python 3.9

        if len(new_set_items) >= min_support:
            new_tid_dict[new_set] = new_set_items
    return new_tid_dict

def filter_min_support(tid_dict, min_support):
    items = dict()
    for item in tqdm(tid_dict, total=len(tid_dict), desc="Filtering min support"):
        if len(tid_dict[item]) >= min_support:
            items[item] = tid_dict[item]
    return items

items = filter_min_support(tid_dict, min_support)

def mine_freq_itemsets(items, min_support):
    new_results = items
    freq_itemsets = items
    i = 1
    while new_results:
        i += 1
        print(f"Mining frequent itemsets of length {i}")
        new_results = combine_sets(new_results, min_support)
        freq_itemsets |= new_results
    return freq_itemsets

freq_itemsets = mine_freq_itemsets(items, min_support)
#freq_itemsets

Filtering min support: 100%|██████████| 11678/11678 [00:00<00:00, 1933107.67it/s]


Mining frequent itemsets of length 2


100%|██████████| 29100630/29100630 [00:25<00:00, 1157023.38it/s]


Mining frequent itemsets of length 3


100%|██████████| 6592056/6592056 [00:04<00:00, 1528049.50it/s]


...

100%|██████████| 133590/133590 [00:00<00:00, 1322797.25it/s]


Mining frequent itemsets of length 5


100%|██████████| 1122/1122 [00:00<00:00, 1123152.53it/s]


Mining frequent itemsets of length 6


100%|██████████| 6/6 [00:00<?, ?it/s]


In [5]:
# Cool, we have quite some frequent itemsets now, let's create some rules based on this
from more_itertools import set_partitions

min_confidence = 0

class Rule:

    def __init__(self, x: frozenset, y: frozenset, confidence: float, sup: float, lift: float):
        self.x = x
        self.y = y
        self.sup = sup
        self.confidence  = confidence
        self.lift = lift

    def __str__(self):
        return f"{set(self.x)} -> {set(self.y)} ({self.confidence:.2f}, {self.sup})"

def get_rules(freq_itemsets, min_confidence):
    rules = list()

    for freq_itemset, tids in freq_itemsets.items():
        n_freq_itemset_transactions = len(tids) # Note that the freq itemset is also #XandY occurances

        # Check all possible bi partitions and see if the generated rule would match the min confidence
        for x, y in set_partitions(freq_itemset, 2):
            x = frozenset(x)
            y = frozenset(y)

            if x not in freq_itemsets:
                # TODO: How come it's not possible that x is in it, but {x, y} is? This should be impossible
                continue # x not in freq_itemset, would mean x (and thus also {x, y}) don't meet the minsup

            n_x_occurances = len(freq_itemsets[x])
            n_y_occurances = len(freq_itemsets[y])
            confidence = n_freq_itemset_transactions / n_x_occurances

            if confidence < min_confidence:
                continue

            p_xy = n_freq_itemset_transactions / len(baskets)
            p_x = n_x_occurances / len(baskets)
            p_y = n_y_occurances / len(baskets)
            lift =  p_xy / (p_x*p_y)

            y = frozenset(y)
            rules.append(Rule(x=x, y=y, confidence=confidence, sup=n_freq_itemset_transactions, lift=lift))
    return rules

rules = get_rules(freq_itemsets, min_confidence)

In [6]:
# Top 10 in terms of support
rules.sort(key=lambda x: (x.sup, x.confidence), reverse=True)
for rule in rules[:10]:
    print(rule)

{5677043} -> {5697463} (0.33, 55)
{5814516} -> {5814517} (0.88, 49)
{5809912} -> {5809910} (0.58, 49)
{5809910} -> {5809911} (0.16, 47)
{5649235} -> {5649236} (0.35, 30)
{5886282} -> {5909810} (0.35, 30)
{5751422} -> {5751383} (0.28, 30)
{5886282} -> {5892179} (0.33, 28)
{5649236} -> {5649271} (0.14, 27)
{5886282} -> {5900651} (0.31, 26)


In [7]:
# Top 10 in terms of lift
rules.sort(key=lambda x: (x.lift, x.sup, x.confidence), reverse=True)
for rule in rules[:10]:
    print(rule)

{5902865} -> {5902863} (1.00, 2)
{5801480} -> {5801481} (1.00, 2)
{5853128} -> {5853126} (1.00, 2)
{5919384} -> {5919382} (1.00, 2)
{5850498} -> {5848914} (1.00, 2)
{5840251} -> {5840213} (1.00, 2)
{5862904} -> {5861441} (1.00, 2)
{5853752} -> {5853751} (1.00, 2)
{5897774} -> {5898382} (1.00, 2)
{5857881} -> {5857882} (1.00, 2)


In [8]:
filtered_rules = list(filter(lambda x: x.confidence > 0.7, rules))
filtered_rules.sort(key=lambda x: (x.sup, x.confidence), reverse=True)
for rule in filtered_rules[:10]:
    print(rule)

{5814516} -> {5814517} (0.88, 49)
{5855091} -> {5855092} (0.82, 18)
{5739056} -> {5739055} (0.74, 17)
{5846436} -> {5846437} (0.74, 17)
{5816166} -> {5809910} (0.78, 14)
{5827356} -> {5827357} (0.92, 12)
{5707826} -> {5692527} (1.00, 9)
{5739122} -> {5739124} (0.82, 9)
{5787479} -> {5692527} (0.75, 9)
{5814516, 5804820} -> {5814517} (0.89, 8)


In [9]:
filtered_rules.sort(key=lambda x: (x.sup*x.confidence, x.sup, x.confidence), reverse=True)
for rule in filtered_rules[:10]:
    print(rule)

{5814516} -> {5814517} (0.88, 49)
{5855091} -> {5855092} (0.82, 18)
{5739056} -> {5739055} (0.74, 17)
{5846436} -> {5846437} (0.74, 17)
{5827356} -> {5827357} (0.92, 12)
{5816166} -> {5809910} (0.78, 14)
{5707826} -> {5692527} (1.00, 9)
{5739122} -> {5739124} (0.82, 9)
{5814516, 5804820} -> {5814517} (0.89, 8)
{5787479} -> {5692527} (0.75, 9)


# Extra information

In [10]:
#dataset = dataset[dataset["event_type"] == "purchase"]
dataset["action_and_product_id"] = dataset["event_type"] + dataset["product_id"].astype(str)
new_baskets = dataset.groupby("user_id").action_and_product_id.apply(set).tolist()
new_baskets[:5]

[{'cart5614842', 'cart5766379', 'purchase5614842', 'purchase5766379'},
 {'cart5861791',
  'cart5894239',
  'purchase5861791',
  'purchase5894239',
  'view5861791',
  'view5894239'},
 {'cart5830270', 'cart5830275', 'purchase5830270', 'remove_from_cart5830275'},
 {'cart5751383',
  'purchase5751383',
  'view5635117',
  'view5751383',
  'view5809910'},
 {'cart5767496',
  'cart5767497',
  'cart5891498',
  'purchase5767496',
  'purchase5767497',
  'purchase5891498',
  'view5891498'}]

In [11]:
def baskets_to_rules(baskets, min_support, min_confidence):
    tid_dict = make_tid_dict(baskets)
    items = filter_min_support(tid_dict, min_support)
    freq_itemsets = mine_freq_itemsets(items, min_support)
    rules = get_rules(freq_itemsets, min_confidence)
    return rules

rules = baskets_to_rules(new_baskets, 3, 0) # Note that I increased the min_support a lot here because computing the rules took way too long with minsup=2

Filtering min support: 100%|██████████| 28110/28110 [00:00<00:00, 2802383.66it/s]


Mining frequent itemsets of length 2


100%|██████████| 43158330/43158330 [00:40<00:00, 1057807.00it/s]


Mining frequent itemsets of length 3


100%|██████████| 45420860/45420860 [00:32<00:00, 1409797.55it/s]


...

100%|██████████| 142506/142506 [00:00<00:00, 1104418.47it/s]


Mining frequent itemsets of length 7


100%|██████████| 4556/4556 [00:00<00:00, 1138675.31it/s]


Mining frequent itemsets of length 8


100%|██████████| 20/20 [00:00<?, ?it/s]


In [17]:
def get_item_nr(action_item):
    # Returns only the item id, e.g. view1234 -> returns 1234
    return int(''.join(char for char in action_item if char.isdigit()))

def all_right_set_items_new_purchases(rule):
    # Checks whether all items in the right set are purchases and that this item didn't appear with another action on the left side
    x, y = rule.x, rule.y

    x_itemids = set()
    for item in x:
        x_itemids.add(get_item_nr(item))

    for item in y:
        if not "purchase" in item or get_item_nr(item) in x_itemids:
            return False
    return True

filtered_rules = list(filter(lambda x: x.confidence > 0.7, rules)) # Applying confidence afterwards and not in the baskets_to_rules function, so it's less computationally intensive to play around with it
filtered_rules = list(filter(lambda rule: all_right_set_items_new_purchases(rule), filtered_rules))
filtered_rules.sort(key=lambda x: (x.sup*x.confidence, x.sup, x.confidence), reverse=True)

for rule in filtered_rules[:10]:
    print(rule)

{'purchase5814516'} -> {'purchase5814517'} (0.90, 46)
{'purchase5814516', 'cart5814516'} -> {'purchase5814517'} (0.90, 43)
{'cart5814516'} -> {'purchase5814517'} (0.86, 43)
{'cart5809911'} -> {'purchase5809910'} (0.78, 36)
{'purchase5819894'} -> {'purchase5823667'} (0.82, 23)
{'purchase5814516', 'purchase5814518'} -> {'purchase5814517'} (1.00, 18)
{'purchase5814518'} -> {'purchase5814517'} (0.86, 19)
{'purchase5855091'} -> {'purchase5855092'} (1.00, 16)
{'cart5814516', 'purchase5814518'} -> {'purchase5814517'} (1.00, 16)
{'purchase5855091', 'cart5855091'} -> {'purchase5855092'} (1.00, 16)
